```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{PyHamcrest: Check What You Want to Check}
\author{Moshe Zadka -- https://cobordism.com}
\date{2020}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

In [69]:
from PIL import Image
import httpx
import collections
import operator
import datetime

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Unit tests and Assertions

### Rough definition of unit tests

### Why assert?

### Bad test one: False alarm

### Bad test two: Missing alarm

### Thinking of unit test value

## PyHamcrest Simple Examples

### Equality

### Sequence containment

### Combination: any

### Combination: all

### Composability

## PyHamcrest Precision

### Order vs. Unorder

### Arbitrary boolean expressions

### Floating point

### String checking

### Dictionary checking

## Custom Matchers

### Concept

### Simple example: prime number

### Simple example use: prime number

### Example: Iterable contains in order

### Use: Iterable contains in order on dequeu

## Subtle PyHamcrest Examples

### Check output from code

### Check property on object

### Combining assertions

### Checking datastructure equivalence

## Conclusion

```{=latex}
\end{document}
```